In [50]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

nltk.download('words')
words = set(nltk.corpus.words.words())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/wcrossb/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to /Users/wcrossb/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [51]:
df1 = pd.read_csv("stored_vars_hr.csv", lineterminator='\n')
df2 = pd.read_csv("stored_vars_hr_2.csv", lineterminator='\n')
df3 = pd.read_csv("stored_vars_hr_3.csv", lineterminator='\n')
df4 = pd.read_csv("stored_vars_hr_4.csv", lineterminator='\n')
df5 = pd.read_csv("stored_vars_hr_5.csv", lineterminator='\n')


print(df2.shape)
df = pd.concat([df1, df2, df3, df4, df5])
print(df.shape)
print(df1.tail())



(36985, 3)
(383051, 3)
       Unnamed: 0                 date  \
55655          58  2022-05-10 20:23:52   
55656          59  2022-05-10 20:15:07   
55657          60  2022-05-10 20:14:03   
55658          61  2022-05-10 20:07:55   
55659          62  2022-05-10 20:00:32   

                                                   tweet  
55655  @henryshield Well, it's happening. Evidence is...  
55656  #DYK ❓ @RFF’s origins can be traced back to a ...  
55657  @swadeshirajp @livennletlive @rohini_sgh @deep...  
55658  @RepBethVanDuyne @POTUS How can someone who kn...  
55659  @LegalScot @marcuscarslaw1 @_KateForbes Perhap...  


In [52]:
#df = pd.read_csv("stored_vars.csv", lineterminator='\n')

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",str(tweet)) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", str(tweet)) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)
         if w.lower() in words or not w.isalpha())
    return tweet

df['tweet_clean'] = df['tweet'].apply(cleaner)


In [53]:
list1 = []
for i in df['tweet_clean']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

df['sentiment'] = pd.Series(list1)
#df.to_csv("inspect.csv")


In [54]:
#Import csv of keywords derived from michigan survey
df_kws = pd.read_csv("kws.csv")
kw_list = df_kws['Indicator'].values.tolist()
category_list = df_kws['Category'].values.tolist()
category_list = [*set(category_list)]

#Detect which keyword was used for search, and store line by line
df['Indicator'] = np.nan
for term in kw_list:
    print(term)
    df['Indicator'] = np.where(df['tweet_clean'].str.contains(term, case=False) == True, term, df['Indicator'])

#Drop Duplicates and compare how many 
print("Length before drop" , df.shape)
df = df.drop_duplicates(subset='tweet_clean', keep="first")
print("length after drop duplicates" , df.shape)
df = pd.merge(df_kws, df, on='Indicator')
df.to_csv("inspect2.csv")



Better financially
Worse financially
Business conditions
Business environment
State of the economy
Unemployment
Economic Depression
Government Economic Policy
Out of work
Economic policy
Interest rates
Rising prices
Disposable income
Incomes
Family income
Gasoline prices
Own a home
Market value of home
House prices
Inflation
Standard of living
Stock market
Cost of living
Length before drop (383051, 6)
length after drop duplicates (312053, 6)


In [55]:
df= pd.read_csv("inspect2.csv", lineterminator='\n')

#df['sentiment_bool'] = np.where(df['sentiment'] > 0, 1, -1)

#df_p = df.pivot_table(index='date', columns='keyword', values='sentiment', aggfunc='mean')
df_p = df.pivot_table(index='date', columns='Category', values='sentiment', aggfunc='mean')

df_p.to_csv("temp4.csv")


In [59]:
df= pd.read_csv("temp4.csv")

df = df.reset_index()
df_c = df[df['date'].str.contains( "'" )==False ]

df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
df_c = df_c.set_index('date')
agg = df_c.resample('D').mean()

cat_list_roll = []
for cat in category_list:   
    agg[cat +"_roll"] = agg[cat].rolling(15).mean()
    cat_list_roll.append([cat +"_roll"])

print(cat_list_roll)
print(agg.head())

agg.to_csv("wed_export.csv")
import plotly.express as px
fig = px.line(agg, x=agg.index, y=["general_roll","prices_roll","personal_roll","policy_roll"], title='PC Sentiment')
fig.show()

[['prices_roll'], ['personal_roll'], ['general_roll'], ['policy_roll']]
             index   general  personal    policy    prices  prices_roll  \
date                                                                      
2021-10-31   313.5 -0.034807  0.137636  0.100639  0.043069          NaN   
2021-11-01   951.5  0.065231 -0.111532  0.037271 -0.014508          NaN   
2021-11-02  1622.5  0.036275  0.008164  0.128181 -0.139769          NaN   
2021-11-03  2343.0  0.059163  0.074829  0.121478  0.003439          NaN   
2021-11-04  3076.5 -0.144131 -0.081968  0.119835  0.086834          NaN   

            personal_roll  general_roll  policy_roll  
date                                                  
2021-10-31            NaN           NaN          NaN  
2021-11-01            NaN           NaN          NaN  
2021-11-02            NaN           NaN          NaN  
2021-11-03            NaN           NaN          NaN  
2021-11-04            NaN           NaN          NaN  


In [57]:
df_p = pd.read_csv("temp4.csv")

df = df_p.reset_index()
df_c = df[df['date'].str.contains( "'" )==False ]

df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
df_c = df_c.set_index('date')
agg = df_c.resample('M').mean()

fig = px.line(agg, x=agg.index, y=["Better financially"], title='PC Sentiment')
fig.show()

"""
agg.to_csv("temp.csv")

agg = agg.reset_index()

start_date = '2017-01-01'
end_date = '2022-10-9'
mask = (agg['date'] > start_date) & (agg['date'] <= end_date)
agg = agg.loc[mask]

save_date = agg[['date']].copy()

agg = agg.drop(columns = ['date','nan'])
"""



ValueError: All arguments should have the same length. The length of argument `y` is 1, whereas the length of  previously-processed arguments ['date'] is 10

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

agg_full = agg.apply(lambda row: row.fillna(row.mean()), axis=1)

#prepare the input series (x) by detrending and standardizing
x = StandardScaler().fit_transform(agg_full)

#Calculate the first  principal component & output explained variance
pca = PCA(n_components=1)
df_PC = pca.fit_transform(x)
print(pca.explained_variance_ratio_)


df_PC_merge = pd.DataFrame(df_PC, columns = ['Sentiment'])

df1 = pd.concat([save_date, df_PC_merge], axis=1)
df_final = df1.set_index('date')
df_final.to_csv("temp.csv")



[0.26063161]


In [ ]:
import plotly.express as px
df_final.to_csv("export.csv")
df_final['Sentiment'] = df_final['Sentiment']*(-1)
fig = px.line(df_final, x=df_final.index, y=["Sentiment"], title='PC Sentiment')
fig.show()

In [ ]:
#This is used for inspecting the values
start_date = '2017-12-01'
end_date = '2018-01-31'
mask = (df['date'] > start_date) & (df['date'] <= end_date)
df_trim = df.loc[mask]

df_trim = df_trim[df_trim['sentiment']>-0.8]
df_trim = df_trim[['date','tweet_clean','sentiment']].copy()
df_trim.to_csv("inspect_high.csv")